# Requirements
- pytorch 
    - Pytorch install: https://pytorch.org/get-started/locally/ 
- tqdm
- sklearn
- Huggingface Transformer: 
    - pip install transformers 
- **DATASET**:
    - https://github.com/BrownFortress/IntentSlotDatasets
    - We will use **ATIS** only
    
    

# Outline

#### Introduction
- sequence labelling (Slot filling)
- text classification (Intent classification)

#### Preparing text for NN
- word2id
- special tokens 
- Customize Dataset class

#### Split data in batches
- Usage of Dataloader class
- Padding sequences

#### Neural Networks in Pytorch
- Word emdbeddings
- Implementation of an LSTM
- Regularization techniques

#### Train and Test a Neural Network
- Optimizer
- Loss function
- Iteration over batches

#### Hugging face library
- Introduction and Usage
 
 
# References
- RNN: https://d2l.ai/chapter_recurrent-neural-networks/index.html 
- LSTM: https://d2l.ai/chapter_recurrent-modern/lstm.html
- GRU: https://d2l.ai/chapter_recurrent-modern/gru.html


# Sequence Labelling and Text classification tasks

### Sequence Labelling
The sequence labelling task is defined as:
- Given a sequence of tokens $w = {w_1, w_2, ..., w_n}$,
- defining a sequence of labels as $l = {l_1, l_2, ..., l_n}$
- compute the sequence $\hat{l}$ such as $\hat{l} = \underset{l}{\operatorname{argmax}} P(l|w)$ 

In this lab session, we are going to see a particular case of sequence labelling task, which is named as Slot Filling (or Concept tagging). In this notebook, the **segmentation** (with IOB tags) and the **labelling** are done at the same time. \
\
An example is the following: 

| Slot Filling |  |                     |                     |  |  |  |  |
|------------------|----|--------------------------|--------------------------|---|------|---|--------|
| Input sequence:  | on | april                    | first                    | I | want | a | flight |
| Output sequence: | O  | B-depart_date.month_name | B-depart_date.day_number | O | O    | O | O      |

### Text classification
The text classification problem is defined *(similar to Sequence Labelling)* as follows:
- Given a sequence of tokens $w = {w_1, w_2, ..., w_n}$,
- And a set of labels $L$ where $l \in L$
- estimate the label $\hat{l}$ such as $\hat{l} = \underset{l}{\operatorname{argmax}} P(l|w)$ 

The text classification task that we are going to see in this laboratory is named as Intent Classification. The Intent is an additional component of the *semantic frame*. \
\
An example is the following:

| Intent Classification|  |                     |                     |  |  |  |  |
|------------------|----|--------------------------|--------------------------|---|------|---|--------|
| Input sequence:  | on | april                    | first                    | I | want | a | flight |
| Output label: | flight     |


# Dataset
The dataset that we are going to use is ATIS (Airline Travel Information Systems). It is composed of trascriptions of humans asking about flight information.  

### Load the dataset
I have prepared a custom data structure for this dataset. The srtucture is the following:
```json
[
    {
    "utterance": "on april first i need a flight going from phoenix to san diego", 
    "slots": "O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name", 
    "intent": "flight"
    },
    "..."
 ]
```

In [ ]:
# Global variables
import os
device = 'cuda:0' # cuda:0 means we are using the GPU with id 0, if you have multiple GPU
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # Used to report errors on CUDA side
PAD_TOKEN = 0

In [ ]:
import json
from pprint import pprint

def load_data(path):
    '''
        input: path/to/data
        output: json 
    '''
    dataset = []
    with open(path) as f:
        dataset = json.loads(f.read())
    return dataset

tmp_train_raw = load_data(os.path.join('data','ATIS','train.json'))
test_raw = load_data(os.path.join('data','ATIS','test.json'))
print('Train samples:', len(tmp_train_raw))
print('Test samples:', len(test_raw))

pprint(tmp_train_raw[0])

### Create a dev set
In the original split the developement set (dev set) is missing. To train and find the best hyperparameter of our network the dev set is fundamental. Thus, we have to create it starting from the **traning** set. The dev set is usually the 10% of the dataset. \
Possible sampling strategies:
* Take the last n elements of the training set.
* Do a random sampling from the training set.
* Do a stratified sampling from the training set using one or more criteria. (The best way)
    * For further details look [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter

# Firt we get the 10% of dataset, then we compute the percentage of these examples 
# on the training set which is around 11% 
portion = round(((len(tmp_train_raw) + len(test_raw)) * 0.10)/(len(tmp_train_raw)),2)


intents = [x['intent'] for x in tmp_train_raw] # We stratify on intents
count_y = Counter(intents)

Y = []
X = []
mini_Train = []

for id_y, y in enumerate(intents):
    if count_y[y] > 1: # If some intents occure once only, we put them in training
        X.append(tmp_train_raw[id_y])
        Y.append(y)
    else:
        mini_Train.append(tmp_train_raw[id_y])
# Random Stratify
X_train, X_dev, y_train, y_dev = train_test_split(X, Y, test_size=portion, 
                                                    random_state=42, 
                                                    shuffle=True,
                                                    stratify=Y)
X_train.extend(mini_Train)
train_raw = X_train
dev_raw = X_dev

y_test = [x['intent'] for x in test_raw]

# Intent distribution
print('Train:')
pprint({k:round(v/len(y_train),3)*100 for k, v in sorted(Counter(y_train).items())})
print('Dev:'), 
pprint({k:round(v/len(y_dev),3)*100 for k, v in sorted(Counter(y_dev).items())})
print('Test:') 
pprint({k:round(v/len(y_test),3)*100 for k, v in sorted(Counter(y_test).items())})
print('='*89)
# Dataset size
print('TRAIN size:', len(train_raw))
print('DEV size:', len(dev_raw))
print('TEST size:', len(test_raw))

### Corvert words to numbers (word2id)
Neural Netwoks in Pytorch, as in other libraries, work with numbers and vectors. (the value type can be both integers or floats)<br><br>
**Exercise** *(5 minutes)*
* Create a dictionary that maps the words in the training set to numbers.
* Define the Dataset class to easily handle the train, dev and test sets.

We will see later how to conver these indexes into vectors (aka embeddings).

*Add special tokens "pad" and "unk"*


In [ ]:
from collections import Counter

w2id = {'pad':PAD_TOKEN}
slot2id = {'pad':PAD_TOKEN}
intent2id = {}
# Map the words only from the train set
# Map slot and intent labels of train, dev and test set. 'unk' is not needed.


print('# Vocab:', len(w2id)-2) # we remove pad and unk from the count
print('# Slots:', len(slot2id)-1)
print('# Intent:', len(intent2id))

### Lang class
Later we will need to convert those numbers in the original form, so we need to invert those dictionaries. We create a calss named as Lang just for convenience.

In [ ]:
from collections import Counter
class Lang():
    def __init__(self, words, intents, slots, cutoff=0):
        self.word2id = self.w2id(words, cutoff=cutoff, unk=True)
        self.slot2id = self.lab2id(slots)
        self.intent2id = self.lab2id(intents, pad=False)
        self.id2word = {v:k for k, v in self.word2id.items()}
        self.id2slot = {v:k for k, v in self.slot2id.items()}
        self.id2intent = {v:k for k, v in self.intent2id.items()}
        
    def w2id(self, elements, cutoff=None, unk=True):
        vocab = {'pad': PAD_TOKEN}
        if unk:
            vocab['unk'] = len(vocab)
        count = Counter(elements)
        for k, v in count.items():
            if v > cutoff:
                vocab[k] = len(vocab)
        return vocab
    
    def lab2id(self, elements, pad=True):
        vocab = {}
        if pad:
            vocab['pad'] = PAD_TOKEN
        for elem in elements:
                vocab[elem] = len(vocab)
        return vocab

In [ ]:
words = sum([x['utterance'].split() for x in train_raw], []) # No set() since we want to compute 
                                                            # the cutoff
corpus = train_raw + dev_raw + test_raw # We do not wat unk labels, 
                                        # however this depends on the research purpose
slots = set(sum([line['slots'].split() for line in corpus],[]))
intents = set([line['intent'] for line in corpus])

lang = Lang(words, intents, slots, cutoff=0)

### Customize the Dataset class
In Pytorch the Dataset class helps you in handleing the dataset. The mandatory methods are ```__init__, __len__ and __getitem__```. <br>
You can find more details here: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 

In [ ]:
import torch
import torch.utils.data as data

class IntentsAndSlots (data.Dataset):
    # Mandatory methods are __init__, __len__ and __getitem__
    def __init__(self, dataset, lang, unk='unk'):
        self.utterances = []
        self.intents = []
        self.slots = []
        self.unk = unk
        
        for x in dataset:
            self.utterances.append(x['utterance'])
            self.slots.append(x['slots'])
            self.intents.append(x['intent'])

        self.utt_ids = self.mapping_seq(self.utterances, lang.word2id)
        self.slot_ids = self.mapping_seq(self.slots, lang.slot2id)
        self.intent_ids = self.mapping_lab(self.intents, lang.intent2id)

    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, idx):
        utt = torch.Tensor(self.utt_ids[idx])
        slots = torch.Tensor(self.slot_ids[idx])
        intent = self.intent_ids[idx]
        sample = {'utterance': utt, 'slots': slots, 'intent': intent}
        return sample
    
    # Auxiliary methods
    
    def mapping_lab(self, data, mapper):
        return [mapper[x] if x in mapper else mapper[self.unk] for x in data]
    
    def mapping_seq(self, data, mapper): # Map sequences to number
        res = []
        for seq in data:
            tmp_seq = []
            for x in seq.split():
                if x in mapper:
                    tmp_seq.append(mapper[x])
                else:
                    tmp_seq.append(mapper[self.unk])
            res.append(tmp_seq)
        return res


In [ ]:
# Create our datasets
train_dataset = IntentsAndSlots(train_raw, lang)
dev_dataset = IntentsAndSlots(dev_raw, lang)
test_dataset = IntentsAndSlots(test_raw, lang)

## Batches
Batches are used to handle large datasets in the memory. Since the whole dataset cannot fit in GPU memories, we rashuffle the dataset and we split it in small batches that will be processed one at a time.
### Padding
Padding is a strategy to fit sequences of different lengths into a matrix. For instance:

| Right padding|   |    |   |  |  |  |  
|---|----|---|---|---|------|---|
| I | saw| a | unk | with | a | telescope | 
| book | me | a | flight | [pad] | [pad] | [pad] | 

| Left padding|   |    |   |  |  |  |  
|---|----|---|---|---|------|---|
| I | saw| a | unk | with | a | telescope | 
| [pad] | [pad] | [pad] | book | me | a | flight | 



**Exercise** *(10 minuts)* <br> 
Write a function that adds padding on the right. (No need to convert the sentences to numbers)

In [ ]:
# split them by white space
sequences = ['I saw a man with a telescope', 
             'book me a flight', 
             'I want to see the flights from Milan to Ibiza']

### Dataloader
To split the dataset into batches and add padding we will use the DataLoader class. 
```python
DataLoader(Dataset, batch_size=N, collate_fn={custom function}, shuffle=True)
```
*collate_fn* is used to shape the batch in output

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(data):
    def merge(sequences):
        '''
        merge from batch * sent_len to batch * max_len 
        '''
        lengths = [len(seq) for seq in sequences]
        max_len = 1 if max(lengths)==0 else max(lengths)
        # Pad token is zero in our case
        # So we create a matrix full of PAD_TOKEN (i.e. 0) with the shape 
        # batch_size X maximum length of a sequence
        padded_seqs = torch.LongTensor(len(sequences),max_len).fill_(PAD_TOKEN)
        for i, seq in enumerate(sequences):
            end = lengths[i]
            padded_seqs[i, :end] = seq # We copy each sequence into the matrix
        # print(padded_seqs)
        padded_seqs = padded_seqs.detach()  # We remove these tensors from the computational graph
        return padded_seqs, lengths
    # Sort data by seq lengths
    data.sort(key=lambda x: len(x['utterance']), reverse=True) 
    new_item = {}
    for key in data[0].keys():
        new_item[key] = [d[key] for d in data]
    # We just need one length for packed pad seq, since len(utt) == len(slots)
    src_utt, _ = merge(new_item['utterance'])
    y_slots, y_lengths = merge(new_item["slots"])
    intent = torch.LongTensor(new_item["intent"])
    
    src_utt = src_utt.to(device) # We load the Tensor on our seleceted device
    y_slots = y_slots.to(device)
    intent = intent.to(device)
    y_lengths = torch.LongTensor(y_lengths).to(device)
    
    new_item["utterances"] = src_utt
    new_item["intents"] = intent
    new_item["y_slots"] = y_slots
    new_item["slots_len"] = y_lengths
    return new_item
# Dataloader instantiation
train_loader = DataLoader(train_dataset, batch_size=128, collate_fn=collate_fn,  shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, collate_fn=collate_fn)

## Define a neural network in Pytorch
In Pythorch the difinition of a neural network is quite flexible. In ```__init__``` the layer that is going to be used are instantiated. In ```forward```, the achitecture of the neural network is defined. Here you can find all the layers provided by Pytorch https://pytorch.org/docs/stable/nn.html while here you can find the recurrent layers https://pytorch.org/docs/stable/nn.html#recurrent-layers. 

<br><br>
**pack_padded_sequence** and **pad_packed_sequences** are to compress and uncompress sequences in order to remove from the computation the padding embeddings. This reduces the computational cost which means speeding up training and therefore reducing CO2 emission.
 ![](https://i.stack.imgur.com/LPHAs.jpg)





In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ModelIAS(nn.Module):

    def __init__(self, hid_size, out_slot, out_int, emb_size, vocab_len, n_layer=1, pad_index=0):
        super(ModelIAS, self).__init__()
        # hid_size = Hidden size
        # out_slot = number of slots (output size for slot filling)
        # out_int = number of intents (ouput size for intent class)
        # emb_size = word embedding size
        
        self.embedding = nn.Embedding(vocab_len, emb_size, padding_idx=pad_index)
        
        self.utt_encoder = nn.LSTM(emb_size, hid_size, n_layer, bidirectional=False)    
        self.slot_out = nn.Linear(hid_size, out_slot)
        self.intent_out = nn.Linear(hid_size, out_int)
        # Dropout layer How do we apply it?
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, utterance, seq_lengths):
        # utterance.size() = batch_size X seq_len
        utt_emb = self.embedding(utterance) # utt_emb.size() = batch_size X seq_len X emb_size
        utt_emb = utt_emb.permute(1,0,2) # we need seq len first -> seq_len X batch_size X emb_size
        
        # pack_padded_sequence avoid computation over pad tokens reducing the computational cost
        
        packed_input = pack_padded_sequence(utt_emb, seq_lengths.cpu().numpy())
        # Process the batch
        packed_output, (last_hidden, cell) = self.utt_encoder(packed_input) 
        # Unpack the sequence
        utt_encoded, input_sizes = pad_packed_sequence(packed_output)
        # Get the last hidden state
        last_hidden = last_hidden[-1,:,:]
        # Compute slot logits
        slots = self.slot_out(utt_encoded)
        # Compute intent logits
        intent = self.intent_out(last_hidden)
        
        # Slot size: seq_len, batch size, calsses 
        slots = slots.permute(1,2,0) # We need this for computing the loss
        # Slot size: batch_size, classes, seq_len
        return slots, intent

### Function to randomly initialize the weights
This is a generic function that randomly initialize the parameters of RNN networks and linear layers. To dig deep in to this I would suggest you to look at here: https://pytorch.org/docs/master/nn.init.html \
\
*Note: In Pytorch every parameter of the network has a proper name like weight_ih, weight_hh etc.*

In [ ]:
def init_weights(mat):
    for m in mat.modules():
        if type(m) in [nn.GRU, nn.LSTM, nn.RNN]:
            for name, param in m.named_parameters():
                if 'weight_ih' in name:
                    for idx in range(4):
                        mul = param.shape[0]//4
                        torch.nn.init.xavier_uniform_(param[idx*mul:(idx+1)*mul])
                elif 'weight_hh' in name:
                    for idx in range(4):
                        mul = param.shape[0]//4
                        torch.nn.init.orthogonal_(param[idx*mul:(idx+1)*mul])
                elif 'bias' in name:
                    param.data.fill_(0)
        else:
            if type(m) in [nn.Linear]:
                torch.nn.init.uniform_(m.weight, -0.01, 0.01)
                if m.bias != None:
                    m.bias.data.fill_(0.01)

## Training set up
Here we initialize the model and we select the hyperparamters of the neural network. Futhermore, we initialize the optimizer and we select the loss function.\
- You can find further optimization algorithms here: https://pytorch.org/docs/stable/optim.html
- and further loss functions here: https://pytorch.org/docs/stable/nn.html#loss-functions

In [ ]:
import torch.optim as optim

hid_size = 200
emb_size = 300

lr = 0.0001 # learning rate
clip = 5 # Clip the gradient

out_slot = len(lang.slot2id)
out_int = len(lang.intent2id)
vocab_len = len(lang.word2id)

model = ModelIAS(hid_size, out_slot, out_int, emb_size, vocab_len, pad_index=PAD_TOKEN).to(device)
model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion_slots = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)
criterion_intents = nn.CrossEntropyLoss() # Because we do not have the pad token

### Train Loop and Evaluation Loop
We define two functions one for training our model and the other for evaluating it. To compute the performances on the slot filling task we will use the **conll script**, while for the intent classification task we are going to use the **classification_report**.

In [ ]:
from conll import evaluate
from sklearn.metrics import classification_report

def train_loop(data, optimizer, criterion_slots, critenrion_intents, model):
    model.train()
    loss_array = []
    for sample in data:
        optimizer.zero_grad() # Zeroing the gradient
        slots, intent = model(sample['utterances'], sample['slots_len'])
        loss_intent = criterion_intents(intent, sample['intents'])
        loss_slot = criterion_slots(slots, sample['y_slots'])
        loss = loss_intent + loss_slot # In joint training we sum the losses. 
                                       # Is there another way to do that?
        loss_array.append(loss.item())
        loss.backward() # Compute the gradient, deleting the computational graph
        # clip the gradient to avoid explosioning gradients
        # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  
        optimizer.step() # Update the weights
    return loss_array

def eval_loop(data, criterion_slots, criterion_intents, model, lang):
    model.eval()
    loss_array = []
    
    ref_intents = []
    hyp_intents = []
    
    ref_slots = []
    hyp_slots = []
    #softmax = nn.Softmax(dim=1) # Use Softmax if you need the actual probability
    with torch.no_grad(): # It used to avoid the creation of computational graph
        for sample in data:
            slots, intents = model(sample['utterances'], sample['slots_len'])
            loss_intent = criterion_intents(intents, sample['intents'])
            loss_slot = criterion_slots(slots, sample['y_slots'])
            loss = loss_intent + loss_slot 
            loss_array.append(loss.item())
            # Intent inference
            # Get the highest probable class
            out_intents = [lang.id2intent[x] 
                           for x in torch.argmax(intents, dim=1).tolist()] 
            gt_intents = [lang.id2intent[x] for x in sample['intents'].tolist()]
            ref_intents.extend(gt_intents)
            hyp_intents.extend(out_intents)
            
            # Slot inference 
            output_slots = torch.argmax(slots, dim=1)
            for id_seq, seq in enumerate(output_slots):
                length = sample['slots_len'].tolist()[id_seq]
                utt_ids = sample['utterance'][id_seq][:length].tolist()
                gt_ids = sample['y_slots'][id_seq].tolist()
                gt_slots = [lang.id2slot[elem] for elem in gt_ids[:length]]
                utterance = [lang.id2word[elem] for elem in utt_ids]
                to_decode = seq[:length].tolist()
                ref_slots.append([(utterance[id_el], elem) for id_el, elem in enumerate(gt_slots)])
                tmp_seq = []
                for id_el, elem in enumerate(to_decode):
                    tmp_seq.append((utterance[id_el], lang.id2slot[elem]))
                hyp_slots.append(tmp_seq)
    try:            
        results = evaluate(ref_slots, hyp_slots)
    except Exception as ex:
        # Sometimes the model predics a class that is not in REF
        print(ex)
        ref_s = set([x[1] for x in ref_slots])
        hyp_s = set([x[1] for x in hyp_slots])
        print(hyp_s.difference(ref_s))
        
    report_intent = classification_report(ref_intents, hyp_intents, 
                                          zero_division=False, output_dict=True)
    return results, report_intent, loss_array


## Train a neural network
We train a neural network iterating several times over the training set. 
* **epochs**: number of times in which the whole training set is seen by the network
* **early stopping**: keeps controlled the performance of the model on the dev set and interrupts the training when the performance is getting worse
    * **patience**: wait for a number of step before interrupting the training, even though the performance is getting worse. 

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
n_epochs = 200
patience = 3
losses_train = []
losses_dev = []
sampled_epochs = []
best_f1 = 0
for x in tqdm(range(1,n_epochs)):
    loss = train_loop(train_loader, optimizer, criterion_slots, 
                      criterion_intents, model)
    if x % 5 == 0:
        sampled_epochs.append(x)
        losses_train.append(np.asarray(loss).mean())
        results_dev, intent_res, loss_dev = eval_loop(dev_loader, criterion_slots, 
                                                      criterion_intents, model, lang)
        losses_dev.append(np.asarray(loss_dev).mean())
        f1 = results_dev['total']['f']
        
        if f1 > best_f1:
            best_f1 = f1
            patience = 3
        else:
            patience -= 1
        if patience <= 0: # Early stopping with patience
            break # Not nice but it keeps the code clean

results_test, intent_test, _ = eval_loop(test_loader, criterion_slots, 
                                         criterion_intents, model, lang)    
print('Slot F1: ', results_test['total']['f'])
print('Intent Accuracy:', intent_test['accuracy'])


### Plot of the train and valid losses
One of the techniques for debugging a neural network is to check the plot of the loss. If the loss goes smoothly down then the network works corretly, otherwise a deeper analysis is needed. Furthermore, this plot can be useful for deciding the learning rate and the optimizer algorithm.

In [ ]:
plt.figure(num = 3, figsize=(8, 5)).patch.set_facecolor('white')
plt.title('Train and Dev Losses')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.plot(sampled_epochs, losses_train, label='Train loss')
plt.plot(sampled_epochs, losses_dev, label='Dev loss')
plt.legend()
plt.show()

### Multiple runs
To have reliable results on small corpora we have to train and test the model from scratch for several times. At the end, we average the results and we compute the standard deviation.

In [ ]:
hid_size = 200
emb_size = 300

lr = 0.0001 # learning rate
clip = 5 # Clip the gradient

out_slot = len(lang.slot2id)
out_int = len(lang.intent2id)
vocab_len = len(lang.word2id)

runs = 5
slot_f1s, intent_acc = [], []
for x in tqdm(range(0, runs)):
    model = ModelIAS(hid_size, out_slot, out_int, emb_size, 
                     vocab_len, pad_index=PAD_TOKEN).to(device)
    model.apply(init_weights)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion_slots = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)
    criterion_intents = nn.CrossEntropyLoss()
    
    n_epochs = 200
    patience = 3
    losses_train = []
    losses_dev = []
    sampled_epochs = []
    best_f1 = 0
    for x in range(1,n_epochs):
        loss = train_loop(train_loader, optimizer, criterion_slots, 
                          criterion_intents, model)
        if x % 5 == 0:
            sampled_epochs.append(x)
            losses_train.append(np.asarray(loss).mean())
            results_dev, intent_res, loss_dev = eval_loop(dev_loader, criterion_slots, 
                                                          criterion_intents, model, lang)
            losses_dev.append(np.asarray(loss_dev).mean())
            f1 = results_dev['total']['f']

            if f1 > best_f1:
                best_f1 = f1
            else:
                patience -= 1
            if patience <= 0: # Early stoping with patient
                break # Not nice but it keeps the code clean

    results_test, intent_test, _ = eval_loop(test_loader, criterion_slots, 
                                             criterion_intents, model, lang)
    intent_acc.append(intent_test['accuracy'])
    slot_f1s.append(results_test['total']['f'])
slot_f1s = np.asarray(slot_f1s)
intent_acc = np.asarray(intent_acc)
print('Slot F1', round(slot_f1s.mean(),3), '+-', round(slot_f1s.std(),3))
print('Intent Acc', round(intent_acc.mean(), 3), '+-', round(slot_f1s.std(), 3))

 ![](https://huggingface.co/front/assets/huggingface_logo-noborder.svg)
# Hugging Face
Hugging Face is a library that allows you to used large pretrained models in an easy way. This means that you do not need to implement an architeture and train it from scratch. Hugging Face is also a community where people share trained models and code.
<br/><br/>
In Hugging Face there are many different models (https://huggingface.co/models) that you can import and each of them has its own input and output shapes. However, Transformer-based models are usually composed of two parts: 
- **Tokenizer**
- **Architecture/Pretrained model**

The **tokenizers** used by Transformer-based models are different from those we saw in the lab. While for instance Spacy's tokenizer is rule-based and splits the text looking at the punctuation, the goal of Transformer tokenizers is to reduce the vocabulary length by spliting words into subwords. To do this, several algorithms have been proposed. If you are interested in this topic you can find a thoroughly explanation here: https://huggingface.co/docs/transformers/tokenizer_summary  

In [ ]:
# BERT model script from: huggingface.co
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("I saw a man with a telescope", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

# Exercise (2 points)
Modify the baseline architecture Model IAS in an addition way:
- Add bidirectionality
- Add dropout layer

***Dataset to use: ATIS***

# Exercise (4 points)
Fine-tune BERT model using a multi-task learning setting on intent classification and slot filling. 
<br>
You can refer to this paper to have a better understanding of such model: https://arxiv.org/abs/1902.10909

***Dataset to use: ATIS***
